## HW11 数据分析及贝叶斯模型分类器
指导教师：胡俊峰

负责助教：苏亚鲁，李浩然

注意：仅需要提交.ipynb文件，请**不要**将下发压缩包中的其他文件一并交上。

截止日期：4月12日24点

### 第二部分：进阶练习（7分）
    使用贝叶斯模型来为用户推荐电影。

In [1]:
import numpy as np
import pandas as pd

# 读取 ratings 文件
ratings = pd.read_csv('data/ratings2.csv',  encoding='latin-1', usecols=['user_id', 'movie_id', 'rating', 'timestamp'])
# 使用全部数据填写数据矩阵，避免造成用户/电影的缺省
user_movie_matrix = ratings.pivot_table('rating', index='user_id', columns='movie_id', aggfunc='mean').fillna(0)
user_movie_matrix.shape

(6040, 3706)

In [2]:
import random
def transform(x):
    if x>2:
        return 1
    else:
        return 0
def mask(x):
    if x>0 and random.random() > 0.2:
        return x
    else:
        return 0

user_movie_matrix = user_movie_matrix.iloc[:,:].applymap(transform)  # 只有2分以上才认为有推荐的必要
train_data_matrix = user_movie_matrix.iloc[:,:].applymap(mask)  # 随机mask掉0.2的观影记录作为测试集
test_data_matrix = user_movie_matrix.subtract(train_data_matrix)  # 测试集

assert user_movie_matrix.values.reshape(1,-1).sum() == train_data_matrix.values.reshape(1,-1).sum() + test_data_matrix.values.reshape(1,-1).sum()

In [3]:
train_x = train_data_matrix.iloc[:,:]
train_y = train_data_matrix.iloc[:,:50]  # 由于电影数据过多，考虑时间问题，这里选取50部电影进行评测。
print(train_x.shape)
print(train_y.shape)
train_y.head()

(6040, 3706)
(6040, 50)


movie_id,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
user_id,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1


In [4]:
test_y = test_data_matrix.iloc[:,:50] # 由于电影数据过多，考虑时间问题，这里选取50部电影进行评测。
print(test_y.shape)
test_y.head()

(6040, 50)


movie_id,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### 2.1 贝叶斯模型训练（3分）
    请根据朴素贝叶斯的后验概率公式分别计算先验概率和似然概率，并进行拉普拉斯平滑化。函数需要返回先验概率P和条件概率E。
    注：E[i]表示待预测电影有评分的前提下，其他各电影观看过的概率，即 P(x_i=1|c=1)

In [5]:
def train(X, Y):
    alpha = 1
    sum_Y = Y.sum(axis=0)
    P = sum_Y # Laplace smoothing
    E = (X.T @ Y + alpha) / (sum_Y + alpha*2)
    return P,E
P, E = train(train_x, train_y)

#### 2.2 贝叶斯模型预测（4分）
    请根据2.1计算出的结果，使用train_x作为输入特征，计算待预测电影的后验概率，并为每个用户筛选出top-3电影和实际的观影记录比较，使用提供的代码进行评测。
    注：对于当前输入的特征（即观影记录），仅使用数值为1的特征值（即看过的电影）对应的条件概率和待预测电影的先验概率一起计算后验概率，数值为0的特征值不予考虑。

In [11]:
def predict(X,P,E):
    top3 = (X@np.log(E))+np.log(P)
    prediction = np.zeros((X.shape[0], P.shape[0]))
    for i in range(X.shape[0]):
        idx = list(np.argsort(top3.iloc[i])[-3:])
        prediction[i][idx] = 1
    return prediction

In [12]:
prediction = predict(train_x,P,E)
pre_list=prediction.flatten()
y = (train_y+test_y).values.reshape(1,-1).tolist()[0]  # 实际观影记录
assert len(pre_list) == len(y)

In [8]:
from sklearn.metrics import accuracy_score,classification_report
print(classification_report(y,pre_list))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93    280198
           1       0.03      0.02      0.03     21802

    accuracy                           0.87    302000
   macro avg       0.48      0.48      0.48    302000
weighted avg       0.86      0.87      0.87    302000

